# Data Analytics for Finance - Final Project #

**Student Name | Student Number:** Hugo Eusébio | 46267

**Signal Name | Signal Code:** Tax Expense Surprise | tax_gr1a




### Strategy Analysis

In [ ]:
// Importing packages

#r "nuget: FSharp.Stats"
#r "nuget: FSharp.Data"
#r "nuget: Plotly.NET, 2.0.0"
#r "nuget: Plotly.NET.Interactive, 2.0.0"
#r "nuget: Accord"
#r "nuget: Accord.Statistics"
#r "nuget: DiffSharp-lite"

#load "YahooFinance.fsx"
#load "Common.fsx"
#load "Portfolio.fsx"

open System
open FSharp.Data
open Plotly.NET
open Common
open YahooFinance
open Portfolio
open FSharp.Stats
open DiffSharp
open Accord
open Accord.Statistics.Models.Regression.Linear

Installed Packages Accord, 3.8.0 Accord.Statistics, 3.8.0 DiffSharp-lite, 1.0.7 FSharp.Data, 4.2.8 FSharp.Stats, 0.4.5 Plotly.NET, 2.0.0 Plotly.NET.Interactive, 2.0.0

Loading extensions from `Plotly.NET.Interactive.dll`

In [ ]:
// Format style

Formatter.Register(fun (x:obj) (writer: TextWriter) -> fprintfn writer "%120A" x )
Formatter.SetPreferredMimeTypesFor(typeof<obj>, "text/plain")
Formatter.SetPreferredMimeTypesFor(typeof<GenericChart.GenericChart>,"text/html")

In [ ]:
// Loading signal data

let [<Literal>] ResolutionFolder = __SOURCE_DIRECTORY__
Environment.CurrentDirectory <- ResolutionFolder

let [<Literal>] IdAndReturnsFilePath = "data/id_and_return_data.csv"
let [<Literal>] MySignalFilePath = "data/tax_gr1a.csv"
let strategyName = "Tax_Expense_Surprise"

type IdAndReturnsType = 
    CsvProvider<Sample=IdAndReturnsFilePath,
                Schema="obsMain(string)->obsMain=bool,exchMain(string)->exchMain=bool",
                ResolutionFolder=ResolutionFolder>

type MySignalType = 
    CsvProvider<MySignalFilePath,
                ResolutionFolder=ResolutionFolder>

let idAndReturnsCsv = IdAndReturnsType.GetSample()
let mySignalCsv = MySignalType.GetSample()

let idAndReturnsRows = idAndReturnsCsv.Rows |> Seq.toList
let mySignalRows = mySignalCsv.Rows |> Seq.toList

In [ ]:
// Defining support functions

let msfBySecurityIdAndMonth =
    idAndReturnsRows
    |> List.map(fun row -> 
        let id = Other row.Id
        let month = DateTime(row.Eom.Year,row.Eom.Month,1)
        let key = id, month
        key, row)
    |> Map    

let signalBySecurityIdAndMonth =
    mySignalRows
    |> List.choose(fun row -> 
        match row.Signal with
        | None -> None
        | Some signal ->
            let id = Other row.Id
            let month = DateTime(row.Eom.Year,row.Eom.Month,1)
            let key = id, month
            Some (key, signal))
    |> Map

let securitiesByFormationMonth =
    idAndReturnsRows
    |> List.groupBy(fun x -> DateTime(x.Eom.Year, x.Eom.Month,1))
    |> List.map(fun (ym, obsThisMonth) -> 
        let idsThisMonth = [ for x in obsThisMonth do Other x.Id ]
        ym, idsThisMonth)
    |> Map

let getInvestmentUniverse formationMonth =
    match Map.tryFind formationMonth securitiesByFormationMonth with
    | Some securities -> 
        { FormationMonth = formationMonth 
          Securities = securities }
    | None -> failwith $"{formationMonth} is not in the date range"

let getMarketCap (security, formationMonth) =
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> None
    | Some row -> 
        match row.MarketEquity with
        | None -> None
        | Some me -> Some (security, me)

let getSecurityReturn (security, formationMonth) =
    let missingReturn = 0.0
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> security, missingReturn
    | Some x ->  
        match x.Ret with 
        | None -> security, missingReturn
        | Some r -> security, r
        
let getMySignal (securityId, formationMonth) =
    match Map.tryFind (securityId, formationMonth) signalBySecurityIdAndMonth with
    | None -> None
    | Some signal ->
        Some { SecurityId = securityId 
               Signal = signal }
               
let getMySignals (investmentUniverse: InvestmentUniverse) =
    let listOfSecuritySignals =
        investmentUniverse.Securities
        |> List.choose(fun security -> 
            getMySignal (security, investmentUniverse.FormationMonth))    
    
    { FormationMonth = investmentUniverse.FormationMonth 
      Signals = listOfSecuritySignals }


let isObsMain (security, formationMonth) =
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> false
    | Some row -> row.ObsMain

let isPrimarySecurity (security, formationMonth) =
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> false
    | Some row -> row.PrimarySec

let isCommonStock (security, formationMonth) =
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> false
    | Some row -> row.Common

let isExchMain (security, formationMonth) =
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> false
    | Some row -> row.ExchMain

let hasMarketEquity (security, formationMonth) =
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> false
    | Some row -> row.MarketEquity.IsSome

let myFilters securityAndFormationMonth =
    isObsMain securityAndFormationMonth &&
    isPrimarySecurity securityAndFormationMonth &&
    isCommonStock securityAndFormationMonth &&
    isExchMain securityAndFormationMonth &&
    isExchMain securityAndFormationMonth &&
    hasMarketEquity securityAndFormationMonth

let doMyFilters (universe:InvestmentUniverse) =
    let filtered = 
        universe.Securities
        |> List.map(fun security -> security, universe.FormationMonth)
        |> List.filter myFilters
        |> List.map fst
    { universe with Securities = filtered }

let startSample = 
    idAndReturnsRows
    |> List.map(fun row -> DateTime(row.Eom.Year,row.Eom.Month,1))
    |> List.min

let endSample = 
    let lastMonthWithData = 
        idAndReturnsRows
        |> Seq.map(fun row -> DateTime(row.Eom.Year,row.Eom.Month,1))
        |> Seq.max

    lastMonthWithData.AddMonths(-1) 

let sampleMonths = getSampleMonths (startSample, endSample)

let formStrategy ym =
    ym
    |> getInvestmentUniverse
    |> doMyFilters
    |> getMySignals
    |> assignSignalSort strategyName 3
    |> List.map (giveValueWeights getMarketCap)
    |> List.map (getPortfolioReturn getSecurityReturn) 
let doParallel = true
let portfolios =
    if doParallel then
        sampleMonths
        |> List.toArray
        |> Array.Parallel.map formStrategy
        |> Array.toList
        |> List.collect id
    else
        sampleMonths
        |> List.collect formStrategy

In [ ]:
// Loading Fama-French data
let ff3 = French.getFF3 Frequency.Monthly
let monthlyRiskFreeRate =
    [ for obs in ff3 do 
        let key = DateTime(obs.Date.Year,obs.Date.Month,1)
        key, obs.Rf ]
    |> Map
    
// Converting returns to excess returns
let portfolioExcessReturns =
    portfolios
    |> List.map(fun x -> 
        match Map.tryFind x.YearMonth monthlyRiskFreeRate with 
        | None -> failwith $"Can't find risk-free rate for {x.YearMonth}"
        | Some rf -> { x with Return = x.Return - rf })

In [ ]:
// Computing Market, Long and Long-short portfolios

let vwMktRf_filter =
    let portfolioMonths = 
        portfolioExcessReturns 
        |> List.map(fun x -> x.YearMonth)
    let minYm = portfolioMonths |> List.min
    let maxYm = portfolioMonths |> List.max
    
    [ for x in ff3 do
        if x.Date >= minYm && x.Date <= maxYm then
            { PortfolioId = Named("Mkt-Rf")
              YearMonth = x.Date
              Return = x.MktRf } ]

let vwMktRf =
    vwMktRf_filter
    |> List.map(fun x ->
        match Map.tryFind x.YearMonth monthlyRiskFreeRate with 
        | None -> failwith $"Can't find risk-free rate for {x.YearMonth}"
        | Some rf -> { x with Return = x.Return - rf })

let long_filter = 
    portfolioExcessReturns 
    |> List.filter(fun x -> 
        x.PortfolioId = Indexed {| Name = strategyName; Index = 3 |})

let long = 
    long_filter
    |> List.map (fun x -> 
        { PortfolioId = Named "Long"
          YearMonth = x.YearMonth
          Return = x.Return})

let short_filter = 
    portfolioExcessReturns 
    |> List.filter(fun x -> 
        x.PortfolioId = Indexed {| Name = strategyName; Index = 1 |})

let short = 
    short_filter
    |> List.map (fun x -> 
        { PortfolioId = Named "Short"
          YearMonth = x.YearMonth
          Return = x.Return})

let longShort = 
    let shortByYearMonthMap = 
        short 
        |> List.map(fun row -> row.YearMonth, row) 
        |> Map
    
    [ for longObs in long do
        match Map.tryFind longObs.YearMonth shortByYearMonthMap with
        | None -> failwith "probably your date variables are not aligned for a weird reason"
        | Some shortObs ->
            { PortfolioId = Named "Long-Short"
              YearMonth = longObs.YearMonth
              Return = longObs.Return - shortObs.Return } ] 

In [ ]:
// Computing Leveraged portfolios

let annualizeMonthlyStdDev monthlyStdDev: float = sqrt(12.0) * monthlyStdDev

let get_stdDevAnnualized (input: PortfolioReturn list) = 
    input
    |> Seq.stDevBy (fun x -> x.Return)
    |> annualizeMonthlyStdDev

let stdDevLongAnnualized = get_stdDevAnnualized long
let stdDevLongShortAnnualized = get_stdDevAnnualized longShort
let stdDevVwMktRfAnnualized = get_stdDevAnnualized vwMktRf

let get_leverage input: float =
    0.1 / input

let leverageLong = get_leverage stdDevLongAnnualized
let leverageLongShort = get_leverage stdDevLongShortAnnualized
let leverageVwMktRf = get_leverage stdDevVwMktRfAnnualized

let longLev_map = 
    long
    |> List.map(fun (x) ->
        { PortfolioId = x.PortfolioId;
          YearMonth = x.YearMonth;
          Return = leverageLong * x.Return })

let longLev = 
    longLev_map
    |> List.map (fun x -> 
        { PortfolioId = Named "Long"
          YearMonth = x.YearMonth
          Return = x.Return })

let longShortLev_map = 
    longShort
    |> List.map(fun (x) ->
        { PortfolioId = x.PortfolioId;
          YearMonth = x.YearMonth;
          Return = leverageLongShort * x.Return })

let longShortLev = 
    longShortLev_map
    |> List.map (fun x -> 
        { PortfolioId = Named "Long-Short"
          YearMonth = x.YearMonth
          Return = x.Return })

let vwMktRfLev_map = 
    vwMktRf
    |> List.map(fun (x) ->
        { PortfolioId = x.PortfolioId;
          YearMonth = x.YearMonth;
          Return = leverageVwMktRf * x.Return })

let vwMktRfLev = 
    vwMktRfLev_map
    |> List.map (fun x -> 
        { PortfolioId = Named "Mkt-Rf"
          YearMonth = x.YearMonth
          Return = x.Return })

In [ ]:
// Preparing portfolios returns plots

let cumulateSimpleReturn (xs: PortfolioReturn list) =
    let accumulator (priorObs:PortfolioReturn) (thisObs:PortfolioReturn) =
        let asOfNow = (1.0 + priorObs.Return)*(1.0 + thisObs.Return) - 1.0
        { thisObs with Return = asOfNow}
    match xs |> List.sortBy(fun x -> x.YearMonth) with
    | [] -> []      
    | head::tail -> 
        (head, tail) 
        ||> List.scan accumulator
        
let portfolioReturnPlot (xs: PortfolioReturn list) =
    xs
    |> List.map(fun x -> x.YearMonth, x.Return)
    |> Chart.Line 
    |> Chart.withLegendStyle(Orientation=StyleParam.Orientation.Vertical)

//Standard portfolios 
let combinedChart =
    List.concat [long; longShort; vwMktRf]
    |> List.groupBy(fun x -> x.PortfolioId)
    |> List.map(fun (portId, xs) ->
        xs
        |> cumulateSimpleReturn
        |> portfolioReturnPlot
        |> Chart.withTitle "Growth of 1 Euro"
        |> Chart.withTraceInfo (Name=portId.ToString()))
    |> Chart.combine
            |> Chart.withSize(600.0,600.0)
          
//Leveraged portfolios plot
let combinedChartLev =
    List.concat [longLev; longShortLev; vwMktRfLev]
    |> List.groupBy(fun x -> x.PortfolioId)
    |> List.map(fun (portId, xs) ->
        xs
        |> cumulateSimpleReturn
        |> portfolioReturnPlot
        |> Chart.withTitle "Growth of 1 Euro with Leverage"
        |> Chart.withTraceInfo (Name=portId.ToString() + " Leveraged"))
    |> Chart.combine
        |> Chart.withSize(600.0,600.0)

In [ ]:
//Standard portfolios plot
combinedChart

<!-- Plotly chart will be drawn inside this DIV --> 


 
</div

In [ ]:
// Leveraged portfolios plot
combinedChartLev

<!-- Plotly chart will be drawn inside this DIV --> 


 
</div

In [ ]:
// Spliting data to calculate performance indicators

let split_list (input_list: list<'a>) = 
    let length =
        input_list
        |> List.length

    let length_proper =
        int(length) / 2

    let firstList = input_list |> Seq.take length_proper |> Seq.toList
    let secondList = input_list |> Seq.skip length_proper |> Seq.toList
    let answer_list = [firstList; secondList]
    answer_list

let longBottomHalf = (split_list long)[0]
let longTopHalf = (split_list long)[1]
let longShortBottomHalf = (split_list longShort)[0]
let longShortTopHalf = (split_list longShort)[1]
let vwMktRfBottomHalf = (split_list vwMktRf)[0]
let vwMktRfTopHalf = (split_list vwMktRf)[1]

In [ ]:
//Computing Annualized Excess Returns

let annualizeMonthlyReturns monthlyReturn = 12.0 * monthlyReturn

let get_avg (input:PortfolioReturn List) = 
    let x = 
        input
        |> List.map (fun x -> x.Return)
        |> List.map ( fun x -> annualizeMonthlyReturns x)
        |> List.average
    let y = 
        x * 100.0
    y

//Long
let longAvg = get_avg long
let longBottomHalfAvg =  get_avg longBottomHalf
let longTopHalfAvg =  get_avg longTopHalf

//Long_Short
let longShortAvg = get_avg longShort
let longShortBottomHalfAvg = get_avg longShortBottomHalf
let longShortTopHalfAvg = get_avg longShortTopHalf

//Market
let vwMktRfAvg = get_avg vwMktRf
let vwMktRfBottomHalfAvg = get_avg vwMktRfBottomHalf
let vwMktRfTopHalfAvg = get_avg vwMktRfTopHalf


In [ ]:
// Long Portfolio Annualized Excess Returns
longAvg, longBottomHalfAvg, longTopHalfAvg

(6.975316875, -0.4230327464, 14.31494944)


In [ ]:
// Long-Short Portfolio Annualized Excess Returns
longShortAvg, longShortBottomHalfAvg, longShortTopHalfAvg

(-0.4153531099, -0.09233533958, -0.7358072471)


In [ ]:
// Market Portfolio Annualized Excess Returns
vwMktRfAvg, vwMktRfBottomHalfAvg, vwMktRfTopHalfAvg

(5.283824701, -4.30944, 14.80095238)


In [ ]:
// Computing Annualized Sharpe Ratios

let sharpe (xs: float seq) =
    (Seq.mean xs) / (Seq.stDev xs)

let annualizeMonthlySharpe monthlySharpe = sqrt(12.0) * monthlySharpe

let get_sharpe (input: PortfolioReturn List) = 
    input
    |> List.map (fun x -> x.Return)
    |> sharpe
    |> annualizeMonthlySharpe

//Long
let longSharpe = get_sharpe long
let longBottomSharpe = get_sharpe longBottomHalf
let longTopSharpe = get_sharpe longTopHalf

//Long-Short
let longShortSharpe = get_sharpe longShort
let longShortBottomSharpe = get_sharpe longShortBottomHalf
let longShortTopSharpe = get_sharpe longShortTopHalf

//Market
let vwMktRfSharpe = get_sharpe vwMktRf
let vwMktRfBottomSharpe = get_sharpe vwMktRfBottomHalf
let vwMktRfTopSharpe = get_sharpe vwMktRfTopHalf

In [ ]:
// Long Portfolio Annualized Sharpe Ratios
longSharpe, longBottomSharpe, longTopSharpe

(0.4382763778, -0.02527235909, 0.9660831117)


In [ ]:
// Long-Short portfolio Annualized Sharpe Ratios
longShortSharpe, longShortBottomSharpe, longShortTopSharpe

(-0.06289872428, -0.01094824278, -0.1804031288)


In [ ]:
// Market portfolio Annualized Sharpe Ratios
vwMktRfSharpe, vwMktRfBottomSharpe, vwMktRfTopSharpe

(0.3320677147, -0.2570009056, 1.016111804)


In [ ]:
// Preparing data for regression

let ff3ByMonth = 
    ff3
    |> Array.map(fun x -> DateTime(x.Date.Year, x.Date.Month,1), x)
    |> Map

type RegData =
    { Date : DateTime
      Portfolio : float
      MktRf : float 
      Hml : float 
      Smb : float }
      
let RegData (input: PortfolioReturn list) = 
    input 
    |> List.map(fun port ->
        let monthToFind = DateTime(port.YearMonth.Year,port.YearMonth.Month,1)
        match Map.tryFind monthToFind ff3ByMonth with
        | None -> failwith "probably you messed up your days of months"
        | Some ff3 -> 
            { Date = monthToFind
              Portfolio = port.Return
              MktRf = ff3.MktRf 
              Hml = ff3.Hml 
              Smb = ff3.Smb })
    |> List.toArray

let capmModelData (input: RegData array) = 
    input
    |> Array.map(fun obs -> [|obs.MktRf|], obs.Portfolio)
    |> Array.unzip 

let ff3ModelData (input: RegData array) =
    input
    |> Array.map(fun obs -> [|obs.MktRf; obs.Hml; obs.Smb |], obs.Portfolio)
    |> Array.unzip

type RegressionOutput =
    { Model : MultipleLinearRegression 
      TValuesWeights : float array
      TValuesIntercept : float 
      R2: float }

type XY = (float array) array * float array

let fitModel (x: (float array) array, y: float array) =
    let ols = new OrdinaryLeastSquares(UseIntercept=true)
    let estimate = ols.Learn(x,y)
    let mse = estimate.GetStandardError(x,y)
    let se = estimate.GetStandardErrors(mse, ols.GetInformationMatrix())
    let tvaluesWeights = 
        estimate.Weights
        |> Array.mapi(fun i w -> w / se.[i])
    let tvalueIntercept = estimate.Intercept / (se |> Array.last)
    let r2 = estimate.CoefficientOfDetermination(x,y)
    { Model = estimate
      TValuesWeights = tvaluesWeights
      TValuesIntercept = tvalueIntercept  
      R2 = r2 }
      
let capmEstimate (input: PortfolioReturn list) =
    RegData input
    |> capmModelData
    |> fitModel

let ff3Estimate (input: PortfolioReturn list) = 
    RegData input
    |> ff3ModelData
    |> fitModel

In [ ]:
//Regressions

//CAPM 
let capmEstimateLong = capmEstimate long
let capmEstimateLongBottom = capmEstimate longBottomHalf
let capmEstimateLongTop = capmEstimate longTopHalf

let capmEstimateLongShort = capmEstimate longShort
let capmEstimateLongShortBottom = capmEstimate longShortBottomHalf
let capmEstimateLongShortTop = capmEstimate longShortTopHalf

//FF3
let ff3EstimateLong = ff3Estimate long
let ff3EstimateLongBottom = ff3Estimate longBottomHalf
let ff3EstimateLongTop = ff3Estimate longTopHalf

let ff3EstimateLongShort = ff3Estimate longShort
let ff3EstimateLongShortBottom = ff3Estimate longShortBottomHalf
let ff3EstimateLongShortTop = ff3Estimate longShortTopHalf

In [ ]:
// Full sample: Long Portfolio CAPM (t-stat, alpha)
capmEstimateLong.TValuesIntercept, capmEstimateLong.Model.Intercept

(0.3542636927, 0.0002326148034)


In [ ]:
// Bottom sample: Long Portfolio CAPM (t-stat, alpha)
capmEstimateLongBottom.TValuesIntercept, capmEstimateLongBottom.Model.Intercept

(0.9299721344, 0.00104478937)


In [ ]:
// Top sample: Long Portfolio CAPM (t-stat, alpha)
capmEstimateLongTop.TValuesIntercept, capmEstimateLongTop.Model.Intercept

(-1.302143229, -0.0008979939711)


In [ ]:
// Full sample: Long-short Portfolio CAPM (t-stat, alpha)
capmEstimateLongShort.TValuesIntercept, capmEstimateLongShort.Model.Intercept

(-0.09492625583, -0.0001147879526)


In [ ]:
// Bottom sample: Long-short Portfolio CAPM (t-stat, alpha)
capmEstimateLongShortBottom.TValuesIntercept, capmEstimateLongShortBottom.Model.Intercept

(-0.1051739979, -0.0002252471229)


In [ ]:
// Top sample: Long-short Portfolio CAPM (t-stat, alpha)
capmEstimateLongShortTop.TValuesIntercept, capmEstimateLongShortTop.Model.Intercept

(-1.038335995, -0.001131442162)


In [ ]:
// Full sample: Long portfolio FF3 (t-stat, alpha)
ff3EstimateLong.TValuesIntercept, ff3EstimateLong.Model.Intercept

(0.5038685885, 0.0003082754874)


In [ ]:
// Bottom sample: Long portfolio FF3 (t-stat, alpha)
ff3EstimateLongBottom.TValuesIntercept, ff3EstimateLongBottom.Model.Intercept

(2.083797519, 0.002135329788)


In [ ]:
// Top sample: Long portfolio FF3 (t-stat, alpha)
ff3EstimateLongTop.TValuesIntercept, ff3EstimateLongTop.Model.Intercept

(-1.88280968, -0.001310754003)


In [ ]:
// Full sample: Long-Short portfolio FF3 (t-stat, alpha)
ff3EstimateLongShort.TValuesIntercept, ff3EstimateLongShort.Model.Intercept

(0.0104881282, 1.152925786e-05)


In [ ]:
// Bottom sample: Long-Short portfolio FF3 (t-stat, alpha)
ff3EstimateLongShortBottom.TValuesIntercept, ff3EstimateLongShortBottom.Model.Intercept

(0.9868302798, 0.001835480959)


In [ ]:
// Top sample: Long-Short portfolio FF3 (t-stat, alpha)
ff3EstimateLongShortTop.TValuesIntercept, ff3EstimateLongShortTop.Model.Intercept

(-1.56350223, -0.00173473683)


In [ ]:
// Computing Information Ratios

type Prediction = { Label : float; Score : float}

let makePredictions 
    (estimate: MultipleLinearRegression) 
    (x: (float array) array, y: float array) =
    (estimate.Transform(x), y)
    ||> Array.zip
    |> Array.map(fun (score, label) -> { Score = score; Label = label })

let residuals (xs: Prediction array) = xs |> Array.map(fun x -> x.Label - x.Score)

let informationRatio monthlyAlpha (monthlyResiduals: float array) =
    let annualAlpha = 12.0 * monthlyAlpha
    let annualStDev = sqrt(12.0) * (Seq.stDev monthlyResiduals)
    annualAlpha / annualStDev

let capmModelDataReg (input: PortfolioReturn list) =
    RegData input
    |> capmModelData

let ff3ModelDataReg (input: PortfolioReturn list) =
    RegData input
    |> ff3ModelData

let capmInformationRatio (input: PortfolioReturn list)  =
    let x =
        makePredictions (capmEstimate input).Model (capmModelDataReg input)
        |> residuals
    informationRatio (capmEstimate input).Model.Intercept x

let ff3InformationRatio (input: PortfolioReturn list)  =
    let x =
        makePredictions (ff3Estimate input).Model (ff3ModelDataReg input)
        |> residuals
    informationRatio (ff3Estimate input).Model.Intercept x
    
//CAPM
let capmInformationratioLong = capmInformationRatio long
let capmInformationratioLongBottom = capmInformationRatio longBottomHalf
let capmInformationratioLongTop = capmInformationRatio longTopHalf

let capmInformationratioLongShort = capmInformationRatio longShort
let capmInformationratioLongShortBottom = capmInformationRatio longShortBottomHalf
let capmInformationratioLongShortTop = capmInformationRatio longShortTopHalf

//FF3
let ff3InformationratioLong = ff3InformationRatio long
let ff3InformationratioLongBottom = ff3InformationRatio longBottomHalf
let ff3InformationratioLongTop = ff3InformationRatio longTopHalf

let ff3InformationratioLongShort = ff3InformationRatio longShort
let ff3InformationratioLongShortBottom = ff3InformationRatio longShortBottomHalf
let ff3InformationratioLongShortTop = ff3InformationRatio longShortTopHalf

In [ ]:
// Long Portfolio CAPM Information Ratios
capmInformationratioLong, capmInformationratioLongBottom, capmInformationratioLongTop

(0.07821876661, 0.2894409216, -0.4218928681)


In [ ]:
// Long-Short Portfolio CAPM Information Ratios
capmInformationratioLongShort, capmInformationratioLongShortBottom, capmInformationratioLongShortTop

(-0.0209590054, -0.03273394734, -0.3364196358)


In [ ]:
// Long Portfolio FF3 Information Ratios
ff3InformationratioLong, ff3InformationratioLongBottom, ff3InformationratioLongTop

(0.1118292655, 0.6750591375, -0.6344099941)


In [ ]:
// Long-Short Portfolio FF3 Information Ratios
ff3InformationratioLongShort, ff3InformationratioLongShortBottom, ff3InformationratioLongShortTop

(0.002327749138, 0.319689793, -0.5268198113)


### Strategy as Part of a Diversified Portfolio

In [ ]:
// Obtaining VTI and BND data

type StockData =
    { Symbol : string 
      Date : DateTime
      Return : float }
  
let ff3new = French.getFF3 Frequency.Monthly |> Array.toList

let ff3StockData =
    [ 
       ff3new |> List.map(fun x -> {Symbol="HML";Date=x.Date;Return=x.Hml})
       ff3new |> List.map(fun x -> {Symbol="MktRf";Date=x.Date;Return=x.MktRf})
       ff3new |> List.map(fun x -> {Symbol="Smb";Date=x.Date;Return=x.Smb})
    ] |> List.concat
  
let tickers = 
    [ 
        "VTI" // Vanguard Total Stock Market ETF
        "BND" // Vanguard Total Bond Market ETF
    ]

let tickPrices = 
    YahooFinance.PriceHistory(
        tickers,
        startDate = DateTime(2000,1,1),
        interval = Monthly)

// Converting ETF´s prices to returns
let pricesToReturns (symbol, adjPrices: list<PriceObs>) =
    adjPrices
    |> List.sortBy (fun x -> x.Date)
    |> List.pairwise
    |> List.map (fun (day0, day1) ->
        let r = day1.AdjustedClose / day0.AdjustedClose - 1.0 
        { Symbol = symbol 
          Date = day1.Date 
          Return = r })

let tickReturns =
    tickPrices
    |> List.groupBy (fun x -> x.Symbol)
    |> List.collect pricesToReturns
    
let min_date = tickReturns[0].Date

In [ ]:
// Computing diversified Long and Long-Short portfolios

let rf = Map [ for x in ff3new do x.Date, x.Rf ]

let long_new = 
    long
    |> List.map(fun x ->
        { Symbol = "Long"
          Date = x.YearMonth
          Return = x.Return })  

let longShort_new = 
    longShort
    |> List.map(fun x ->
        { Symbol = "Long-Short"
          Date = x.YearMonth
          Return = x.Return })  

let get_standardInvestmentExcess (input: StockData list) =
    let result = 
        let part_1 =
            let maxff3Date = ff3new |> List.map(fun x -> x.Date) |> List.max
            tickReturns
            |> List.filter(fun x -> x.Date <= maxff3Date)
            |> List.map(fun x -> 
                match Map.tryFind x.Date rf with 
                | None -> failwith $"why isn't there a rf for {x.Date}"
                | Some rf -> { x with Return = x.Return - rf })
        let part_2 =
            let maxff3Date = ff3new |> List.map(fun x -> x.Date) |> List.max
            input 
            |> List.filter(fun x -> x.Date >= min_date)
            |> List.filter(fun x -> x.Date <= maxff3Date)
            |> List.map(fun x -> 
                match Map.tryFind x.Date rf with 
                | None -> failwith $"why isn't there a rf for {x.Date}"
                | Some rf -> { x with Return = x.Return })
        part_1 @ part_2
        |> Seq.distinct
        |> List.ofSeq
    result  
    
let standardInvestmentsExcess = get_standardInvestmentExcess long_new
let standardInvestmentsExcessLongShort = get_standardInvestmentExcess longShort_new

In [ ]:
// Computing tickers for our portfolios
let get_stockData (input: StockData list) = 
    input
    |> List.groupBy(fun x -> x.Symbol)
    |> Map

In [ ]:
// Computing tickers for our portfolios

let stockData = get_stockData standardInvestmentsExcess
let stockDataLongShort = get_stockData standardInvestmentsExcessLongShort

let tickers_tweaked =
    [ 
        "VTI" // Vanguard Total Stock Market ETF
        "BND" // Vanguard Total Bond Market ETF
        "Long"
    ]

let tickers_tweaked_LongShort =
    [ 
        "VTI" 
        "BND" 
        "Long-Short"
    ]

let tickers_6040 =
    [ 
        "VTI" 
        "BND" 
    ]

In [ ]:
// Obtaining covariances

let getCov xId yId (stockData: Map<string,StockData list>) =
    let xRet = 
        stockData[xId] 
        |> List.map (fun x -> x.Date,x.Return) 
        |> Map
    let yRet = 
        stockData[yId]
        |> List.map (fun y -> y.Date, y.Return)
        |> Map
    let overlappingDates =
        [ xRet.Keys |> set
          yRet.Keys |> set]
        |> Set.intersectMany
    [ for date in overlappingDates do xRet[date], yRet[date]]
    |> Seq.covOfPairs

// Obtaining weights
let get_weights (input_tickers: seq<string>) (input_stockdata: Map<string,list<StockData>>) =
    let covariances =
        [ for rowTick in input_tickers do 
            [ for colTick in input_tickers do
                getCov rowTick colTick input_stockdata ]]
        |> dsharp.tensor
    let means =
        [ for ticker in input_tickers do 
            input_stockdata[ticker]
            |> List.averageBy (fun x -> x.Return)]
        |> dsharp.tensor
    let w' = dsharp.solve(covariances,means)
    let w = w' / w'.sum()
    let weights =
        Seq.zip input_tickers (w.toArray1D<float>())
        |> Map.ofSeq
    weights  

In [ ]:
//Weights for Long and Long-Short portfolios

let weights = get_weights tickers_tweaked stockData
let weightsLongShort = get_weights tickers_tweaked_LongShort stockDataLongShort

weights, weightsLongShort

(map [("BND", 0.8472900987); ("Long", 0.2642714679); ("VTI", -0.1115615815)],
 map [("BND", 0.7270628214); ("Long-Short", 0.130771637); ("VTI", 0.1421655267)])


In [ ]:
//Computing MVE and 60/40 portfolios

let get_stockDataByDate (input: seq<list<StockData>>) = 
    input
    |> Seq.toList
    |> List.collect id 
    |> List.groupBy(fun x -> x.Date)
    |> List.sortBy fst

In [ ]:
//Computing MVE and 60/40 portfolios

let stockDataByDate = get_stockDataByDate stockData.Values
let stockDataByDateLongShort = get_stockDataByDate stockDataLongShort.Values

let get_AllAssetsStart (input_stockDataByDate:list<DateTime * list<StockData>>) (input_tickers:list<string>) = 
    input_stockDataByDate
    |> List.find(fun (month, stocks) -> stocks.Length = input_tickers.Length)
    |> fst 

let get_AllAssetsEnd (input_stockDataByDate:list<DateTime * list<StockData>>) (input_tickers:list<string>) = 
    input_stockDataByDate
    |> List.findBack(fun (month, stocks) -> stocks.Length = input_tickers.Length)
    |> fst 
    
let allAssetsStart =
    get_AllAssetsStart stockDataByDate tickers_tweaked

let allAssetsStartLongShort =
    get_AllAssetsStart stockDataByDateLongShort tickers_tweaked_LongShort

let allAssetsEnd =
    get_AllAssetsEnd stockDataByDate tickers_tweaked

let allAssetsEndLongShort =
    get_AllAssetsEnd stockDataByDateLongShort tickers_tweaked_LongShort
    
let stockDataByDateComplete =
    stockDataByDate
    |> List.filter(fun (date, stocks) -> 
        date >= allAssetsStart &&
        date <= allAssetsEnd)

let stockDataByDateCompleteLongShort =
    stockDataByDateLongShort
    |> List.filter(fun (date, stocks) -> 
        date >= allAssetsStartLongShort &&
        date <= allAssetsEndLongShort)

In [ ]:
//Computing MVE and 60/40 portfolios

let portfolioMonthReturn (weights: Map<string,float>) (monthData: list<StockData>) =
    weights
    |> Map.toList
    |> List.map(fun (symbol, weight) ->
        let symbolData = 
            match monthData |> List.tryFind(fun x -> x.Symbol = symbol) with
            | None -> failwith $"You tried to find {symbol} in the data but it was not there"
            | Some data -> data
        symbolData.Return*weight)
    |> List.sum

let weights6040 = Map [("VTI",0.6);("BND",0.4)]

let portMve =
    stockDataByDateComplete
    |> List.map(fun (date, data) -> 
        { Symbol = "MVE Long"
          Date = date
          Return = portfolioMonthReturn weights data })

let portMveLongShort =
    stockDataByDateCompleteLongShort
    |> List.map(fun (date, data) -> 
        { Symbol = "MVE Long-Short"
          Date = date
          Return = portfolioMonthReturn weightsLongShort data })

let port6040 = 
    stockDataByDateComplete
    |> List.map(fun (date, data) -> 
        { Symbol = "60/40"
          Date = date 
          Return = portfolioMonthReturn weights6040 data})

In [ ]:
// Computing Portfolio indicators

let get_avg_StockData (input:StockData List) = 
    let x = 
        input
        |> List.map (fun x -> x.Return)
        |> List.map ( fun x -> annualizeMonthlyReturns x)
        |> List.average
    let y = 
        x * 100.0
    y

let get_sharpe_StockData (input: StockData List) = 
    input
    |> List.map (fun x -> x.Return)
    |> sharpe
    |> annualizeMonthlySharpe

let get_stdDevAnnualized_StockData (input: StockData list) = 
    input
    |> Seq.stDevBy (fun x -> x.Return)
    |> annualizeMonthlyStdDev

// Annualized Returns
let MveAvg = get_avg_StockData portMve
let MveLongShortAvg = get_avg_StockData portMveLongShort
let port6040Avg = get_avg_StockData port6040

//Sharpe Ratio´s
let MveSharpe = get_sharpe_StockData portMve
let MveLongShortSharpe = get_sharpe_StockData portMveLongShort
let port6040Sharpe = get_sharpe_StockData port6040

//Standard Deviation
let stdDevMveAnnualized = get_stdDevAnnualized_StockData portMve
let stdDevMveLongShortAnnualized = get_stdDevAnnualized_StockData portMveLongShort
let stdDev6040Annualized = get_stdDevAnnualized_StockData port6040

In [ ]:
// Average Annualized Returns
MveAvg, MveLongShortAvg, port6040Avg

(4.657661418, 4.058223255, 7.215323341)


In [ ]:
// Sharpe Ratio´s
MveSharpe, MveLongShortSharpe, port6040Sharpe

(1.124521239, 1.102921397, 0.7109974372)


In [ ]:
// Computing Leveraged portfolios

let leverageMve = get_leverage stdDevMveAnnualized
let leverageMveLongShort = get_leverage stdDevMveLongShortAnnualized
let leverage6040 = get_leverage stdDev6040Annualized

let MveLev = 
    portMve
    |> List.map(fun (x) ->
        { Symbol = "MVE Long";
          Date = x.Date;
          Return = leverageMve * x.Return })

let MveLongShortLev = 
    portMveLongShort
    |> List.map(fun (x) ->
        { Symbol = "MVE Long-Short";
          Date = x.Date;
          Return = leverageMve * x.Return })

let portf6040Lev = 
    port6040
    |> List.map(fun (x) ->
        { Symbol = "60/40" ;
          Date = x.Date;
          Return = leverage6040 * x.Return })

In [ ]:
// Computing plots

let cumulateSimpleReturn_div (xs: StockData list) =
    let accumulator (priorObs:StockData) (thisObs:StockData) =
        let asOfNow = (1.0 + priorObs.Return)*(1.0 + thisObs.Return) - 1.0
        { thisObs with Return = asOfNow}
    match xs |> List.sortBy(fun x -> x.Date) with
    | [] -> []      
    | head::tail -> 
        (head, tail) 
        ||> List.scan accumulator

let portfolioReturnPlot_div (xs: StockData list) =
    xs
    |> List.map(fun x -> x.Date, x.Return)
    |> Chart.Line 
    |> Chart.withLegendStyle(Orientation=StyleParam.Orientation.Horizontal)
    
let combineMveChart =
    List.concat [portMve; portMveLongShort; port6040]
    |> List.groupBy(fun x -> x.Symbol)
    |> List.map(fun (portId, xs) ->
        xs
        |> cumulateSimpleReturn_div
        |> portfolioReturnPlot_div
        |> Chart.withTitle "MVE & 60/40 Portfolios"
        |> Chart.withTraceInfo (Name=portId.ToString()))
    |> Chart.combine
        |> Chart.withSize(600.0,600.0)

let combinedMveChart_LEV =
    List.concat [MveLev; MveLongShortLev; portf6040Lev]
    |> List.groupBy(fun x -> x.Symbol)
    |> List.map(fun (portId, xs) ->
        xs
        |> cumulateSimpleReturn_div
        |> portfolioReturnPlot_div
        |> Chart.withTitle "MVE & 60/40 Portfolios Leveraged"
        |> Chart.withTraceInfo (Name=portId.ToString() + " Leveraged"))
    |> Chart.combine
        |> Chart.withSize(600.0,600.0)

In [ ]:
combineMveChart

<!-- Plotly chart will be drawn inside this DIV --> 


 
</div

In [ ]:
combinedMveChart_LEV

<!-- Plotly chart will be drawn inside this DIV --> 


 
</div